In [1]:
import MySQLdb
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import Formatter
import math
import ipywidgets as widgets
import xueqiu
import time
import stock

In [6]:
c,kline,kdate = stock.loadKline('SZ002230')
bo = stock.bollLine(kline,20)
vb= stock.bollWidth(bo)
stock.Kline(kline,len(kline)-100,len(kline),kdate=kdate,boll=20,volume=True,macd=vb)

Box(children=(Button(description='上一页', style=ButtonStyle()), Button(description='下一页', style=ButtonStyle())),…

Output()

In [7]:
c,kline,kdate = stock.loadKline('SH600519')
bo = stock.bollLine(kline,20)
vb= stock.bollWidth(bo)
stock.Kline(kline,len(kline)-100,len(kline),kdate=kdate,boll=20,volume=True,macd=vb)

Box(children=(Button(description='上一页', style=ButtonStyle()), Button(description='下一页', style=ButtonStyle())),…

Output()

In [8]:
c,kline,kdate = stock.loadKline('SH601099')
bo = stock.bollLine(kline,20)
vb= stock.bollWidth(bo)
stock.Kline(kline,len(kline)-100,len(kline),kdate=kdate,boll=20,volume=True,macd=vb)

Box(children=(Button(description='上一页', style=ButtonStyle()), Button(description='下一页', style=ButtonStyle())),…

Output()